In [7]:
import pandas as pd
import numpy as np
from math import sqrt
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
# Create directory for Tensorboard
import os, time, datetime
stamp = datetime.datetime.now()
stamp = time.mktime(stamp.timetuple())
cwd = os.getcwd()
PATH_TO_LOG = cwd+'/tfboards/run_'+str(stamp)
os.mkdir(PATH_TO_LOG)

In [18]:
# Specify model for data handling
rnn_flag = False

In [22]:
# Handle data.
df = pd.read_csv('../data/20190606180000-20190606190000.csv')
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna(axis=0);


# columns of interest
cols = ['B:VIMIN', 'B_VIMIN','B:IMINER']

# Extract data we're focused on
df2=df[cols]
#df2=df2.set_index(pd.DatetimeIndex(df2['time'])) # set index to time (not in-place operation)

# clean data: remove rows with spurious values of B:VIMIN (>= 3 stdevs)
std_vimin  = np.std(df['B:VIMIN'])
mean_vimin = np.mean(df['B:VIMIN'])
df2=df2[df2['B:VIMIN']< mean_vimin + 3*std_vimin]
print(df2.shape)

C:\Users\Rachael\.conda\envs\py3tf1gnt\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (77,79,81,83,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(48004, 3)


In [23]:
# FUNCTION: series_to_supervised
# INPUT: data   ~ numpy array of time series observations
#       n_lag   ~ number of lag observations as input 
#       n_out   ~ number of observations as output.
#       dropnan ~ Boolean whether or not to drop rows with NaN values.
# sourced: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
def series_to_supervised(data, n_lag=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    col_data = []
    col_names = []
    # build the input sequence by concatenating farthest to closest in time
    for i in range(n_lag, 0, -1):
        col_data.append(df.shift(i))
        col_names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # build the output sequence
    for i in range(0, n_out):
        col_data.append(df.shift(-i))
        if i == 0:
            col_names += [('x%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            col_names += [('x%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(col_data, axis=1)
    agg.columns =col_names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# select B:VIMIN to convert to lagged time series
cycle  = 15
scaler = MinMaxScaler(feature_range=(0, 1))
df_xy  = pd.DataFrame(scaler.fit_transform(df2.values))
# convert B:VIMIN to lagged series
df_ts = pd.DataFrame(series_to_supervised(list(df_xy.iloc[:,0]),cycle))
df_ts=df_ts.reset_index()

# join lagged time series with the non-lagged features AND target: B_VIMIN (a feature) and target 
data = pd.concat([df_ts, df_xy.iloc[:, 1:]], axis = 1,ignore_index=True)

In [34]:
data= data.drop(data.columns[0],axis=1)

In [48]:
data=data.dropna(axis=0)

In [49]:
p = 0.8 # percent of data to take
n_train = int(len(data)* p)


train = data.sample(n= n_train, random_state = 42)
test = data.drop(train.index)


# split into input and outputs
x_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
x_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]

#L RNN LSTM Model only
if rnn_flag == True:
# reshape input to be 3D [samples, timesteps, features]
    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(38391, 17) (38391,) (9598, 17) (9598,)


In [50]:
l=pd.isna(x_train).any()
data.head()
l.shape

(17,)

In [52]:
# design network
n_hidden1 = 10
n_hidden2 = 10
EPOCHS = 1000
BATCH_SIZE = 1000


model = Sequential()

model.add(Dense(n_hidden1, activation='relu', input_dim=(x_train.shape[1]))) # input to hidden1
model.add(Dense(n_hidden2, activation='relu')) # hidden1 to hidden2
model.add(Dense(1, activation='linear')) # output layer


model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mae'])

# fit network
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_test, y_test), verbose=2, shuffle=False)

Train on 38391 samples, validate on 9598 samples
Epoch 1/1000
 - 0s - loss: 0.0879 - mean_absolute_error: 0.0879 - val_loss: 0.0834 - val_mean_absolute_error: 0.0834
Epoch 2/1000
 - 0s - loss: 0.0799 - mean_absolute_error: 0.0799 - val_loss: 0.0805 - val_mean_absolute_error: 0.0805
Epoch 3/1000
 - 0s - loss: 0.0765 - mean_absolute_error: 0.0765 - val_loss: 0.0775 - val_mean_absolute_error: 0.0775
Epoch 4/1000
 - 0s - loss: 0.0737 - mean_absolute_error: 0.0737 - val_loss: 0.0738 - val_mean_absolute_error: 0.0738
Epoch 5/1000
 - 0s - loss: 0.0712 - mean_absolute_error: 0.0712 - val_loss: 0.0712 - val_mean_absolute_error: 0.0712
Epoch 6/1000
 - 0s - loss: 0.0692 - mean_absolute_error: 0.0692 - val_loss: 0.0698 - val_mean_absolute_error: 0.0698
Epoch 7/1000
 - 0s - loss: 0.0676 - mean_absolute_error: 0.0676 - val_loss: 0.0678 - val_mean_absolute_error: 0.0678
Epoch 8/1000
 - 0s - loss: 0.0661 - mean_absolute_error: 0.0661 - val_loss: 0.0665 - val_mean_absolute_error: 0.0665
Epoch 9/1000
 -

Epoch 71/1000
 - 0s - loss: 0.0526 - mean_absolute_error: 0.0526 - val_loss: 0.0515 - val_mean_absolute_error: 0.0515
Epoch 72/1000
 - 0s - loss: 0.0526 - mean_absolute_error: 0.0526 - val_loss: 0.0515 - val_mean_absolute_error: 0.0515
Epoch 73/1000
 - 0s - loss: 0.0526 - mean_absolute_error: 0.0526 - val_loss: 0.0515 - val_mean_absolute_error: 0.0515
Epoch 74/1000
 - 0s - loss: 0.0525 - mean_absolute_error: 0.0525 - val_loss: 0.0514 - val_mean_absolute_error: 0.0514
Epoch 75/1000
 - 0s - loss: 0.0525 - mean_absolute_error: 0.0525 - val_loss: 0.0514 - val_mean_absolute_error: 0.0514
Epoch 76/1000
 - 0s - loss: 0.0524 - mean_absolute_error: 0.0524 - val_loss: 0.0514 - val_mean_absolute_error: 0.0514
Epoch 77/1000
 - 0s - loss: 0.0524 - mean_absolute_error: 0.0524 - val_loss: 0.0513 - val_mean_absolute_error: 0.0513
Epoch 78/1000
 - 0s - loss: 0.0524 - mean_absolute_error: 0.0524 - val_loss: 0.0513 - val_mean_absolute_error: 0.0513
Epoch 79/1000
 - 0s - loss: 0.0523 - mean_absolute_error

 - 0s - loss: 0.0514 - mean_absolute_error: 0.0514 - val_loss: 0.0509 - val_mean_absolute_error: 0.0509
Epoch 141/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0506 - val_mean_absolute_error: 0.0506
Epoch 142/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0508 - val_mean_absolute_error: 0.0508
Epoch 143/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0507 - val_mean_absolute_error: 0.0507
Epoch 144/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0507 - val_mean_absolute_error: 0.0507
Epoch 145/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0507 - val_mean_absolute_error: 0.0507
Epoch 146/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0506 - val_mean_absolute_error: 0.0506
Epoch 147/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.0513 - val_loss: 0.0506 - val_mean_absolute_error: 0.0506
Epoch 148/1000
 - 0s - loss: 0.0513 - mean_absolute_error: 0.05

Epoch 209/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 210/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0503 - val_mean_absolute_error: 0.0503
Epoch 211/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 212/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 213/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0505 - val_mean_absolute_error: 0.0505
Epoch 214/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0503 - val_mean_absolute_error: 0.0503
Epoch 215/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 216/1000
 - 0s - loss: 0.0508 - mean_absolute_error: 0.0508 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 217/1000
 - 0s - loss: 0.0508 - mean_absol

Epoch 278/1000
 - 0s - loss: 0.0506 - mean_absolute_error: 0.0506 - val_loss: 0.0502 - val_mean_absolute_error: 0.0502
Epoch 279/1000
 - 0s - loss: 0.0506 - mean_absolute_error: 0.0506 - val_loss: 0.0503 - val_mean_absolute_error: 0.0503
Epoch 280/1000
 - 0s - loss: 0.0505 - mean_absolute_error: 0.0505 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 281/1000
 - 0s - loss: 0.0505 - mean_absolute_error: 0.0505 - val_loss: 0.0504 - val_mean_absolute_error: 0.0504
Epoch 282/1000
 - 0s - loss: 0.0505 - mean_absolute_error: 0.0505 - val_loss: 0.0500 - val_mean_absolute_error: 0.0500
Epoch 283/1000
 - 0s - loss: 0.0505 - mean_absolute_error: 0.0505 - val_loss: 0.0500 - val_mean_absolute_error: 0.0500
Epoch 284/1000
 - 0s - loss: 0.0505 - mean_absolute_error: 0.0505 - val_loss: 0.0503 - val_mean_absolute_error: 0.0503
Epoch 285/1000
 - 0s - loss: 0.0505 - mean_absolute_error: 0.0505 - val_loss: 0.0501 - val_mean_absolute_error: 0.0501
Epoch 286/1000
 - 0s - loss: 0.0505 - mean_absol

...

Epoch 692/1000
 - 0s - loss: 0.0497 - mean_absolute_error: 0.0497 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 693/1000
 - 0s - loss: 0.0497 - mean_absolute_error: 0.0497 - val_loss: 0.0499 - val_mean_absolute_error: 0.0499
Epoch 694/1000
 - 0s - loss: 0.0497 - mean_absolute_error: 0.0497 - val_loss: 0.0493 - val_mean_absolute_error: 0.0493
Epoch 695/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0502 - val_mean_absolute_error: 0.0502
Epoch 696/1000
 - 0s - loss: 0.0497 - mean_absolute_error: 0.0497 - val_loss: 0.0493 - val_mean_absolute_error: 0.0493
Epoch 697/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0493 - val_mean_absolute_error: 0.0493
Epoch 698/1000
 - 0s - loss: 0.0497 - mean_absolute_error: 0.0497 - val_loss: 0.0500 - val_mean_absolute_error: 0.0500
Epoch 699/1000
 - 0s - loss: 0.0497 - mean_absolute_error: 0.0497 - val_loss: 0.0499 - val_mean_absolute_error: 0.0499
Epoch 700/1000
 - 0s - loss: 0.0497 - mean_absol

Epoch 761/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0494 - val_mean_absolute_error: 0.0494
Epoch 762/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0495 - val_mean_absolute_error: 0.0495
Epoch 763/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 764/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0494 - val_mean_absolute_error: 0.0494
Epoch 765/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 766/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0494 - val_mean_absolute_error: 0.0494
Epoch 767/1000
 - 0s - loss: 0.0496 - mean_absolute_error: 0.0496 - val_loss: 0.0494 - val_mean_absolute_error: 0.0494
Epoch 768/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 769/1000
 - 0s - loss: 0.0496 - mean_absol

Epoch 830/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0494 - val_mean_absolute_error: 0.0494
Epoch 831/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 832/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0493 - val_mean_absolute_error: 0.0493
Epoch 833/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 834/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 835/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 836/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 837/1000
 - 0s - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 838/1000
 - 0s - loss: 0.0495 - mean_absol

Epoch 899/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 900/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 901/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 902/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 903/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 904/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 905/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 906/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0492 - val_mean_absolute_error: 0.0492
Epoch 907/1000
 - 0s - loss: 0.0494 - mean_absol

Epoch 968/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 969/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 970/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 971/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 972/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 973/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 974/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 975/1000
 - 0s - loss: 0.0494 - mean_absolute_error: 0.0494 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 976/1000
 - 0s - loss: 0.0494 - mean_absol